In [ ]:
 pip install -U scikit-fuzzy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894079 sha256=924948a89e0f7207df821c1d2d035d289cb9a9dc77dfec205040b3339f33f228
  Stored in directory: /root/.cache/pip/wheels/4f/86/1b/dfd97134a2c8313e519bcebd95d3fedc7be7944db022094bc8
Successfully built scikit-fuzzy


In [ ]:
import pandas as pd
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import random

df = pd.read_csv("kz.csv")
df = df.head(1000)

category_popularity = df.groupby('category_id').size().reset_index(name='popularity')

user_preference = ctrl.Antecedent(np.arange(0, 11, 1), 'User Preference')
item_popularity = ctrl.Antecedent(np.arange(0, category_popularity['popularity'].max() + 1, 1), 'Item Popularity')
recommendation_score = ctrl.Consequent(np.arange(0, 101, 1), 'Recommendation Score')

user_preference.automf(3)
item_popularity.automf(3)
recommendation_score.automf(5)

rule1 = ctrl.Rule(user_preference['poor'] | item_popularity['poor'], recommendation_score['average'])
rule2 = ctrl.Rule(user_preference['average'] | item_popularity['average'], recommendation_score['decent'])
rule3 = ctrl.Rule(user_preference['good'] | item_popularity['good'], recommendation_score['good'])
rule4 = ctrl.Rule(user_preference['poor'] | item_popularity['good'], recommendation_score['mediocre'])
rule5 = ctrl.Rule(user_preference['good'] | item_popularity['poor'], recommendation_score['decent'])

recommendation_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5])

recommendation_system = ctrl.ControlSystemSimulation(recommendation_ctrl)

random_category = random.choice(category_popularity['category_id'].values)

random_user_preference = random.randint(0, 10)

category_popularity_value = category_popularity[category_popularity['category_id'] == random_category]['popularity'].values[0]

recommendation_system.input['User Preference'] = random_user_preference
recommendation_system.input['Item Popularity'] = category_popularity_value

recommendation_system.compute()

recommendation_score = recommendation_system.output['Recommendation Score']

print(f'Random Category ID: {random_category}')
print(f'Recommendation Accuracy: {recommendation_score:.2f}%')

Random Category ID: 2.2681054044854692e+18
Recommendation Accuracy: 65.45%
